# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# !pip install snorkel

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('../all_sqlbelle_youtube_comments_4200rows.csv')

In [ ]:
df_train.textDisplay[:-3:-1]

4201                              Thank you George :)
4200    Awesome! Looking forward to many more videos.
Name: textDisplay, dtype: object

In [ ]:
df_train.tail(2)

,response_kind,vid_channel_id,vid_id,vid_published_at,vid_title,vid_desc,vid_chan_title,is_live_brodcast,vid_publish_time,view_count,...,canRate,viewerRating,likeCount,publishedAt,updatedAt,tcomment_id,reply_id,canReply,totalReplyCount,isPublic
4200,youtube#commentThread,NaN,ZxcWBnXY8ps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,none,1.0,2020-09-02T06:16:13Z,2020-09-02T06:16:13Z,UgzO3LCmld1HWW3XpNt4AaABAg,NaN,True,1.0,True
4201,youtube#comment,NaN,ZxcWBnXY8ps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,True,none,0.0,2020-09-03T05:16:12Z,2020-09-03T05:16:12Z,UgzO3LCmld1HWW3XpNt4AaABAg,UgzO3LCmld1HWW3XpNt4AaABAg.9D5QmTk8TpI9D7thkihuok,NaN,NaN,NaN


#### Create simple labels to start

In [ ]:
abstain = -1
not_spam = 0
spam = 1

#### Key Word:  simple function to identify comments using "my"

In [ ]:
from snorkel.labeling import labeling_function


@labeling_function()
def lf_keyword_my(x):
    """Many spam comments talk about 'my channel', 'my video', etc."""
    return spam if "my" in x.text.lower() else abstain

#### Regular Expression: simple fuction

In [ ]:
import re


@labeling_function()
def lf_regex_check_out(x):
    """Spam comments say 'check out my video', 'check it out', etc."""
    return spam if re.search(r"check.*out", x.text, flags=re.I) else abstain

#### Arbitrary Heuristics

In [ ]:
@labeling_function()
def lf_short_comment(x):
    """Non-spam comments are often short, such as 'cool video!'."""
    return not_spam if len(x.text.split()) < 5 else abstain

#### Third Party Models:
>
    -We use a third-party sentiment classification model, TextBlob.

    -We combine this with the heuristic that non-spam comments are often positive.

In [ ]:
# !pip install textblob

In [ ]:
from textblob import TextBlob


@labeling_function()
def lf_textblob_polarity(x):
    
    return not_spam if TextBlob(x.text).sentiment.polarity > 0.3 else abstain

# Combining & Cleaning the Labels

In [ ]:
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier

In [ ]:
df = df_train
#create toy dataset
df =df[['textDisplay']]
#drop na's
df = df.loc[~df.textDisplay.isna()]
# create string and 
df = df.textDisplay.astype(str)

df = df.reset_index()
#rename columns
df.rename(columns = {'index': 'num','textDisplay':'text'},inplace=True)
df.tail(2)

df_train = df
df_train.head(3)

,num,text
0,45,Thank you. Very clear and informative
1,46,Really good content. Looking forward to an ad...
2,47,"Hello Belle, I am having a problem, when I am ..."


In [ ]:
# Define the set of labeling functions (LFs)
lfs = [lf_keyword_my, lf_regex_check_out, lf_short_comment, lf_textblob_polarity]

# Apply the LFs to the unlabeled training data
applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

100%|██████████| 4156/4156 [00:02<00:00, 1406.38it/s]
INFO:root:Computing O...
INFO:root:Estimating \mu...
 59%|█████▉    | 296/500 [00:00<00:00, 470.59epoch/s]INFO:root:[300 epochs]: TRAIN:[loss=0.001]
INFO:root:[350 epochs]: TRAIN:[loss=0.000]
100%|██████████| 500/500 [00:01<00:00, 409.81epoch/s]
INFO:root:Finished Training


#### Filter out all data that was abstained from labeling

In [ ]:
df_train = df_train[df_train.label != abstain]

#### Augmentation Transfomers
>Create functions that use nltk synonyms add more versions of our data with synonyms for model training

In [ ]:
import random

import nltk
from nltk.corpus import wordnet as wn

from snorkel.augmentation import transformation_function

nltk.download("wordnet", quiet=True)


def get_synonyms(word):
    """Get the synonyms of word from Wordnet."""
    lemmas = set().union(*[s.lemmas() for s in wn.synsets(word)])
    return list(set(l.name().lower().replace("_", " ") for l in lemmas) - {word})


@transformation_function()
def tf_replace_word_with_synonym(x):
    """Try to replace a random word with a synonym."""
    words = x.text.lower().split()
    idx = random.choice(range(len(words)))
    synonyms = get_synonyms(words[idx])
    if len(synonyms) > 0:
        x.text = " ".join(words[:idx] + [synonyms[0]] + words[idx + 1 :])
        return x

#### Apply these functions to our dataset

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/davidramsey/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from snorkel.augmentation import ApplyOnePolicy, PandasTFApplier

tf_policy = ApplyOnePolicy(n_per_original=2, keep_original=True)
tf_applier = PandasTFApplier([tf_replace_word_with_synonym], tf_policy)
df_train_augmented = tf_applier.apply(df_train)

100%|██████████| 2738/2738 [00:07<00:00, 367.27it/s]


### Writing a Slicing Function
> create functions used to get rid of or "slice" out unwanted data

In [ ]:
from snorkel.slicing import slicing_function


@slicing_function()
def short_link(x):
    """Return whether text matches common pattern for shortened ".ly" links."""
    return int(bool(re.search(r"\w+\.ly", x.text)))

### Train a Classifier

In [ ]:
df_train

,num,text,label
1,46,Really good content. Looking forward to an ad...,0
3,48,Do u hv any udemy course belle??,0
4,49,Thank you s o much! you are awesome teacher,0
10,55,recommend this for anyone who want to learn ta...,0
11,56,Thank you Michal! :),1
...,...,...,...
4149,4195,Thank you ahdrful!,1
4151,4197,Glad to hear there's already a new tidbit you ...,0
4152,4198,"love the video! Every easy to follow, can't wa...",0
4154,4200,Awesome! Looking forward to many more videos.,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

train_text = df_train_augmented.text.tolist()
X_train = CountVectorizer(ngram_range=(1, 2)).fit_transform(train_text)

clf = LogisticRegression(solver="lbfgs")
clf.fit(X=X_train, y=df_train_augmented.label.values)

LogisticRegression()

In [ ]:
df_train

,num,text,label
1,46,Really good content. Looking forward to an ad...,0
3,48,Do u hv any udemy course belle??,0
4,49,Thank you s o much! you are awesome teacher,0
10,55,recommend this for anyone who want to learn ta...,0
11,56,Thank you Michal! :),1
...,...,...,...
4149,4195,Thank you ahdrful!,1
4151,4197,Glad to hear there's already a new tidbit you ...,0
4152,4198,"love the video! Every easy to follow, can't wa...",0
4154,4200,Awesome! Looking forward to many more videos.,0


In [ ]:
df_train_augmented

,num,text,label
1,46,Really good content. Looking forward to an ad...,0
1,46,really practiced content. looking forward to a...,0
1,46,really good content. looking forward to an adv...,0
3,48,Do u hv any udemy course belle??,0
3,48,come u hv any udemy course belle??,0
...,...,...,...
4152,4198,"love the video! Every easy to follow, can't wa...",0
4154,4200,Awesome! Looking forward to many more videos.,0
4154,4200,awesome! looking forrad to many more videos.,0
4155,4201,Thank you George :),1


### Lets check to see what was labeled as SPAM

In [ ]:
df_train_augmented[df_train_augmented.label ==1]

,num,text,label
11,56,Thank you Michal! :),1
13,58,Thank you Suganthi!,1
13,58,give thanks you suganthi!,1
13,58,give thanks you suganthi!,1
17,62,Thank you Guna!,1
...,...,...,...
4145,4191,give thanks you mustapha!,1
4149,4195,Thank you ahdrful!,1
4149,4195,give thanks you ahdrful!,1
4155,4201,Thank you George :),1


In [ ]:
#| export
def foo(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()